### Import libraries

In [88]:
import pandas as pd
import numpy as np
import folium
import requests
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

### Read html of list of postal codes of Canada to DataFrame

In [32]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.rename(columns = {'Neighbourhood': 'Neighborhood'}, inplace = True)
df.head()

Postal Code           Borough               Neighborhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront

### Process DataFrame values 

In [33]:
data = df[df['Borough'] != 'Not assigned']
data.head()

Postal Code           Borough                                 Neighborhood
2         M3A        North York                                    Parkwoods
3         M4A        North York                             Victoria Village
4         M5A  Downtown Toronto                    Regent Park, Harbourfront
5         M6A        North York             Lawrence Manor, Lawrence Heights
6         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

In [34]:
data2 = data.groupby('Postal Code')['Postal Code'].value_counts()
data2

Postal Code  Postal Code
M1B          M1B            1
M1C          M1C            1
M1E          M1E            1
M1G          M1G            1
M1H          M1H            1
                           ..
M9N          M9N            1
M9P          M9P            1
M9R          M9R            1
M9V          M9V            1
M9W          M9W            1
Name: Postal Code, Length: 103, dtype: int64

In [36]:
data[data['Neighborhood'] == 'Not assigned']['Neighborhood'] = data['Borough']
data.head()

Postal Code           Borough                                 Neighborhood
2         M3A        North York                                    Parkwoods
3         M4A        North York                             Victoria Village
4         M5A  Downtown Toronto                    Regent Park, Harbourfront
5         M6A        North York             Lawrence Manor, Lawrence Heights
6         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

In [37]:
data.shape

(103, 3)

In [38]:
### Get latitude and longitude from geocoder AP

In [39]:
import geocoder

In [40]:
def get_lat_long_cords(postal_code):
    lat_long_cords = None
    while lat_long_cords is None:
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_long_cords = g.latlng
        
    return lat_long_cords[0], lat_long_cords[1]

In [41]:
data2 = data.copy()

In [42]:
data2[['Latitude', 'Longitude']] = data2.apply(lambda x: get_lat_long_cords(x['Postal Code']) , axis = 1, result_type="expand")
data2.head()

Postal Code           Borough                                 Neighborhood  \
2         M3A        North York                                    Parkwoods   
3         M4A        North York                             Victoria Village   
4         M5A  Downtown Toronto                    Regent Park, Harbourfront   
5         M6A        North York             Lawrence Manor, Lawrence Heights   
6         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

   Latitude  Longitude  
2  43.75245  -79.32991  
3  43.73057  -79.31306  
4  43.65512  -79.36264  
5  43.72327  -79.45042  
6  43.66253  -79.39188

### Filtering for Toronto Data

In [43]:
data_tor = data2[data2['Borough'].str.contains('Toronto')]
data_tor.head()

Postal Code           Borough                                 Neighborhood  \
4          M5A  Downtown Toronto                    Regent Park, Harbourfront   
6          M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
13         M5B  Downtown Toronto                     Garden District, Ryerson   
22         M5C  Downtown Toronto                               St. James Town   
30         M4E      East Toronto                                  The Beaches   

    Latitude  Longitude  
4   43.65512  -79.36264  
6   43.66253  -79.39188  
13  43.65739  -79.37804  
22  43.65215  -79.37587  
30  43.67709  -79.29547

In [44]:

map_toronto = folium.Map(location=[data_tor['Latitude'].mean(), data_tor['Longitude'].mean()], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(data_tor['Latitude'], data_tor['Longitude'], data_tor['Borough'], data_tor['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
#map_toronto

In [45]:
from sklearn.cluster import KMeans

### Credentials Foursquare

In [53]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

### Function to get nearby venues for each neighborhood

In [54]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [58]:
venues_tor = getNearbyVenues(names=data_tor['Neighborhood'], latitudes=data_tor['Latitude'], longitudes=data_tor['Longitude'])
venues_tor.head()

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  Regent Park, Harbourfront               43.65512               -79.36264   
1  Regent Park, Harbourfront               43.65512               -79.36264   
2  Regent Park, Harbourfront               43.65512               -79.36264   
3  Regent Park, Harbourfront               43.65512               -79.36264   
4  Regent Park, Harbourfront               43.65512               -79.36264   

                    Venue  Venue Latitude  Venue Longitude  Venue Category  
0        Roselle Desserts       43.653447       -79.362017          Bakery  
1           Tandem Coffee       43.653559       -79.361809     Coffee Shop  
2  Figs Breakfast & Lunch       43.655675       -79.364503  Breakfast Spot  
3         The Yoga Lounge       43.655515       -79.364955     Yoga Studio  
4     Body Blitz Spa East       43.654735       -79.359874             Spa

In [60]:
venues_tor.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Berczy Park                                                            60   
Brockton, Parkdale Village, Exhibition Place                           85   
Business reply mail Processing Centre, South Ce...                    100   
CN Tower, King and Spadina, Railway Lands, Harb...                     77   
Central Bay Street                                                     76   
Christie                                                               11   
Church and Wellesley                                                   79   
Commerce Court, Victoria Hotel                                        100   
Davisville                                                             26   
Davisville North                                                        8   
Dufferin, Dovercourt Village                                           17   
First Canadian Place, Underground city                                100   
Forest Hill North & West, Forest Hill Road Park                         2   
Garden District, Ryerson                                              100   
Harbourfront East, Union Station, Toronto Islands                      58   
High Park, The Junction South                                           4   
India Bazaar, The Beaches West                                         18   
Kensington Market, Chinatown, Grange Park                              57   
Lawrence Park                                                           2   
Little Portugal, Trinity                                               45   
Moore Park, Summerhill East                                             3   
North Toronto West, Lawrence Park                                       3   
Parkdale, Roncesvalles                                                 53   
Queen's Park, Ontario Provincial Government                            18   
Regent Park, Harbourfront                                              20   
Richmond, Adelaide, King                                              100   
Rosedale                                                                5   
Runnymede, Swansea                                                     38   
St. James Town                                                         83   
St. James Town, Cabbagetown                                            48   
Stn A PO Boxes                                                        100   
Studio District                                                        51   
Summerhill West, Rathnelly, South Hill, Forest ...                      6   
The Annex, North Midtown, Yorkville                                    21   
The Beaches                                                             4   
The Danforth West, Riverdale                                            8   
Toronto Dominion Centre, Design Exchange                              100   
University of Toronto, Harbord                                         53   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Berczy Park                                                             60   
Brockton, Parkdale Village, Exhibition Place                            85   
Business reply mail Processing Centre, South Ce...                     100   
CN Tower, King and Spadina, Railway Lands, Harb...                      77   
Central Bay Street                                                      76   
Christie                                                                11   
Church and Wellesley                                                    79   
Commerce Court, Victoria Hotel                                         100   
Davisville                                                              26   
Davisville North                                                         8   
Dufferin, Dovercourt Village       

In [65]:
venues_tor.head()

Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  Regent Park, Harbourfront               43.65512               -79.36264   
1  Regent Park, Harbourfront               43.65512               -79.36264   
2  Regent Park, Harbourfront               43.65512               -79.36264   
3  Regent Park, Harbourfront               43.65512               -79.36264   
4  Regent Park, Harbourfront               43.65512               -79.36264   

                    Venue  Venue Latitude  Venue Longitude  Venue Category  
0        Roselle Desserts       43.653447       -79.362017          Bakery  
1           Tandem Coffee       43.653559       -79.361809     Coffee Shop  
2  Figs Breakfast & Lunch       43.655675       -79.364503  Breakfast Spot  
3         The Yoga Lounge       43.655515       -79.364955     Yoga Studio  
4     Body Blitz Spa East       43.654735       -79.359874             Spa

In [67]:
tor_onehot = pd.get_dummies(venues_tor[['Venue Category']], prefix="", prefix_sep="")
tor_onehot.head()

Accessories Store  Afghan Restaurant  American Restaurant  Antique Shop  \
0                  0                  0                    0             0   
1                  0                  0                    0             0   
2                  0                  0                    0             0   
3                  0                  0                    0             0   
4                  0                  0                    0             0   

   Aquarium  Art Gallery  Art Museum  Arts & Crafts Store  Asian Restaurant  \
0         0            0           0                    0                 0   
1         0            0           0                    0                 0   
2         0            0           0                    0                 0   
3         0            0           0                    0                 0   
4         0            0           0                    0                 0   

   Athletics & Sports  ...  Train Station  Vegetarian / Vegan Restaurant  \
0                   0  ...              0                              0   
1                   0  ...              0                              0   
2                   0  ...              0                              0   
3                   0  ...              0                              0   
4                   0  ...              0                              0   

   Veterinarian  Video Game Store  Vietnamese Restaurant  Wine Bar  Wine Shop  \
0             0                 0                      0         0          0   
1             0                 0                      0         0          0   
2             0                 0                      0         0          0   
3             0                 0                      0         0          0   
4             0                 0                      0         0          0   

   Wings Joint  Women's Store  Yoga Studio  
0            0              0            0  
1            0              0            0  
2            0              0            0  
3            0              0            1  
4            0              0            0  

[5 rows x 229 columns]

In [77]:
tor_onehot['Neighborhood'] = venues_tor['Neighborhood'] 
prev_list = list(tor_onehot.columns)
prev_list.remove('Neighborhood')
columns_list = ['Neighborhood'] + prev_list
tor_onehot = tor_onehot[columns_list]
print(tor_onehot.shape)
tor_onehot.head()

(1739, 229)


Neighborhood  Accessories Store  Afghan Restaurant  \
0  Regent Park, Harbourfront                  0                  0   
1  Regent Park, Harbourfront                  0                  0   
2  Regent Park, Harbourfront                  0                  0   
3  Regent Park, Harbourfront                  0                  0   
4  Regent Park, Harbourfront                  0                  0   

   American Restaurant  Antique Shop  Aquarium  Art Gallery  Art Museum  \
0                    0             0         0            0           0   
1                    0             0         0            0           0   
2                    0             0         0            0           0   
3                    0             0         0            0           0   
4                    0             0         0            0           0   

   Arts & Crafts Store  Asian Restaurant  ...  Train Station  \
0                    0                 0  ...              0   
1                    0                 0  ...              0   
2                    0                 0  ...              0   
3                    0                 0  ...              0   
4                    0                 0  ...              0   

   Vegetarian / Vegan Restaurant  Veterinarian  Video Game Store  \
0                              0             0                 0   
1                              0             0                 0   
2                              0             0                 0   
3                              0             0                 0   
4                              0             0                 0   

   Vietnamese Restaurant  Wine Bar  Wine Shop  Wings Joint  Women's Store  \
0                      0         0          0            0              0   
1                      0         0          0            0              0   
2                      0         0          0            0              0   
3                      0         0          0            0              0   
4                      0         0          0            0              0   

   Yoga Studio  
0            0  
1            0  
2            0  
3            1  
4            0  

[5 rows x 229 columns]

In [80]:
tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()
print(tor_grouped.shape)
tor_grouped.head()

(38, 229)


Neighborhood  Accessories Store  \
0                                        Berczy Park           0.000000   
1       Brockton, Parkdale Village, Exhibition Place           0.011765   
2  Business reply mail Processing Centre, South C...           0.000000   
3  CN Tower, King and Spadina, Railway Lands, Har...           0.000000   
4                                 Central Bay Street           0.000000   

   Afghan Restaurant  American Restaurant  Antique Shop  Aquarium  \
0                0.0                 0.00      0.016667       0.0   
1                0.0                 0.00      0.000000       0.0   
2                0.0                 0.02      0.000000       0.0   
3                0.0                 0.00      0.000000       0.0   
4                0.0                 0.00      0.000000       0.0   

   Art Gallery  Art Museum  Arts & Crafts Store  Asian Restaurant  ...  \
0     0.016667    0.000000             0.000000          0.000000  ...   
1     0.000000    0.000000             0.023529          0.011765  ...   
2     0.010000    0.000000             0.010000          0.030000  ...   
3     0.000000    0.000000             0.000000          0.012987  ...   
4     0.013158    0.013158             0.000000          0.000000  ...   

   Train Station  Vegetarian / Vegan Restaurant  Veterinarian  \
0       0.000000                       0.016667      0.000000   
1       0.000000                       0.011765      0.000000   
2       0.000000                       0.020000      0.000000   
3       0.012987                       0.000000      0.012987   
4       0.000000                       0.000000      0.000000   

   Video Game Store  Vietnamese Restaurant  Wine Bar  Wine Shop  Wings Joint  \
0          0.000000               0.000000  0.000000        0.0          0.0   
1          0.000000               0.000000  0.000000        0.0          0.0   
2          0.000000               0.000000  0.010000        0.0          0.0   
3          0.000000               0.000000  0.000000        0.0          0.0   
4          0.013158               0.013158  0.013158        0.0          0.0   

   Women's Store  Yoga Studio  
0            0.0     0.016667  
1            0.0     0.011765  
2            0.0     0.000000  
3            0.0     0.012987  
4            0.0     0.000000  

[5 rows x 229 columns]

In [84]:
# set number of clusters
kclusters = 5

tor_grouped_clustering = tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [94]:
# add clustering labels
tor_grouped.insert(0, 'Cluster Labels', kmeans.labels_)
tor_grouped['Cluster Labels'] = tor_grouped['Cluster Labels'].astype('int')
tor_merged = data_tor

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(tor_grouped.set_index('Neighborhood'), on='Neighborhood')

tor_merged.head() # check the last columns!

ValueError: cannot insert Cluster Labels, already exists

In [97]:
tor_merged['Cluster Labels'].unique()
tor_merged2 = tor_merged[~tor_merged['Cluster Labels'].isna()]

In [98]:
# create map
map_clusters = folium.Map(location=[data_tor['Latitude'].mean(), data_tor['Longitude'].mean()], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged2['Latitude'], tor_merged2['Longitude'], tor_merged2['Neighborhood'], tor_merged2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters